In [1]:
!pip install opencv-python
!pip install pytesseract
!pip install transformers
!pip install datasets
!pip install onnxruntime
!pip install onnx
!pip install onnxruntime-tools
!apt-get install -y poppler-utils
!apt-get install tesseract-ocr
!pip install accelerate -U
!pip install transformers[torch]
!pip install datasets
!pip install onnxruntime
!pip install onnx
!pip install onnxruntime-tools
!apt-get install -y poppler-utils
!apt-get install tesseract-ocr
!pip install pdf2image
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
import pytesseract
from pdf2image import convert_from_path
import os
import re
from transformers import LayoutLMForTokenClassification, LayoutLMTokenizer, TrainingArguments, Trainer
from datasets import load_metric, Dataset
import torch
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType
import onnxruntime as ort
import numpy as np


In [3]:
# Function to convert PDF to text
def pdf_to_text(pdf_path):
    try:
        images = convert_from_path(pdf_path)
        text = ""
        for img in images:
            text += pytesseract.image_to_string(img)
        return text
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        return ""

# Process multiple PDF files
pdf_files = ["/content/invoice_Chad Cunningham_1935.pdf", "/content/invoice_Matt Connell_28870.pdf", "/content/invoice_Troy Staebel_30585.pdf", "/content/invoice_Xylona Preis_15577.pdf", "/content/invoice_Zuschuss Carroll_15953.pdf"]  # Replace with your PDF file paths
texts = []
for pdf_file in pdf_files:
    texts.append(pdf_to_text(pdf_file))

# Check extracted text
for i, text in enumerate(texts):
    print(f"Text from {pdf_files[i]}:\n{text}\n")


Text from /content/invoice_Chad Cunningham_1935.pdf:
Superstore INVOICE

# 1935
Date: Feb 03 2013
Bill To: Ship To:
— pe . Ship Mode: Second Class
Chad Cunningham Guasave, Sinaloa,

Mexico Balance Due: $1,867.70
heya Quantity latclic) Amount
HP Fax Machine, Laser 3 $598.68 $1,796.04
Copiers, Technology, TEC-CO-4776

Subtotal: $1,796.04
Discount (0.2%): $3.59
Shipping: $75.25
Total: $1,867.70

Notes:
Thanks for your business!

Terms:
Order ID : MX-2013-CC1210082-41308


Text from /content/invoice_Matt Connell_28870.pdf:
Superstore INVOICE

# 28870
Date: Dec 11 2012
Bill To: Ship To: .
Ship Mode: Standard Class
Matt Connell Perth, Western
Australia, Australia Balance Due: $3,110.79
heya Quantity latclic) Amount
Bush Classic Bookcase, Metal 3 $1,112.78 $3,338.33
Bookcases, Furniture, FUR-BO-3624
Subtotal: $3,338.33
Discount (10%): $333.83
Shipping: $106.29
Total: $3,110.79

Notes:

Thanks for your business!

Terms:
Order ID : IN-2012-MC176057-41254


Text from /content/invoice_Troy Stae

In [4]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n', ' ', text)
    return text.strip()

def annotate_text(text):
    annotations = {}
    annotations['invoice_number'] = re.findall(r'Invoice Number: (\d+)', text)
    annotations['invoice_date'] = re.findall(r'Date: (\d{2}/\d{2}/\d{4})', text)
    annotations['total_amount'] = re.findall(r'Total: \$([0-9,.]+)', text)
    return annotations

preprocessed_texts = [preprocess_text(text) for text in texts]
annotations = [annotate_text(text) for text in preprocessed_texts]

for i, annotation in enumerate(annotations):
    print(f"Annotations from {pdf_files[i]}:\n{annotation}\n")


Annotations from /content/invoice_Chad Cunningham_1935.pdf:
{'invoice_number': [], 'invoice_date': [], 'total_amount': ['1,867.70']}

Annotations from /content/invoice_Matt Connell_28870.pdf:
{'invoice_number': [], 'invoice_date': [], 'total_amount': ['3,110.79']}

Annotations from /content/invoice_Troy Staebel_30585.pdf:
{'invoice_number': [], 'invoice_date': [], 'total_amount': ['7,773.25']}

Annotations from /content/invoice_Xylona Preis_15577.pdf:
{'invoice_number': [], 'invoice_date': [], 'total_amount': ['1,717.28']}

Annotations from /content/invoice_Zuschuss Carroll_15953.pdf:
{'invoice_number': [], 'invoice_date': [], 'total_amount': ['17,560.21']}



In [7]:
!pip install transformers datasets accelerate torch


In [9]:
def encode_labels(text, annotations):
    tokens = tokenizer(text, truncation=True, padding='max_length', max_length=512)
    labels = [0] * len(tokens['input_ids'])  # Initialize all labels to 0
    for idx in range(len(tokens['input_ids'])):
        if tokens['input_ids'][idx] == tokenizer.convert_tokens_to_ids('invoice'):
            labels[idx] = 1
    tokens['labels'] = labels
    return tokens

def encode_examples(examples):
    return encode_labels(examples['text'], examples['annotations'])

# Sample data
texts = ["Sample invoice text one", "Sample invoice text two"]
annotations = [{'invoice_number': ['123'], 'invoice_date': ['01/01/2021'], 'total_amount': ['$100']},
               {'invoice_number': ['456'], 'invoice_date': ['02/02/2021'], 'total_amount': ['$200']}]

# Preprocess texts
preprocessed_texts = [text.strip() for text in texts]

# Create dataset dictionary
dataset_dict = {'text': preprocessed_texts, 'annotations': annotations}

# Convert to Hugging Face dataset
dataset = Dataset.from_dict(dataset_dict)

# Encode dataset
encoded_dataset = dataset.map(lambda e: encode_examples(e), batched=True)
encoded_dataset = encoded_dataset.remove_columns(["text", "annotations"])
encoded_dataset.set_format("torch")


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [10]:
model_name = "microsoft/layoutlm-base-uncased"
model = LayoutLMForTokenClassification.from_pretrained(model_name, num_labels=2)
tokenizer = LayoutLMTokenizer.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Adjusted batch size
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,
)


Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def export_to_onnx(model, tokenizer, onnx_path):
    model.eval()
    dummy_input = tokenizer("Example input", return_tensors="pt")
    torch.onnx.export(model, (dummy_input['input_ids'],), onnx_path,
                      input_names=["input_ids"], output_names=["output"])

onnx_path = "invoice_model.onnx"
export_to_onnx(model, tokenizer, onnx_path)

# Quantize the ONNX model
quantized_model_path = "invoice_model_quantized.onnx"
quantize_dynamic(onnx_path, quantized_model_path, weight_type=QuantType.QUInt8)


In [16]:
model_name = "microsoft/layoutlm-base-uncased"
tokenizer = LayoutLMTokenizer.from_pretrained(model_name)

# Assuming you have already converted your model to ONNX and it's saved as layoutlm.onnx
onnx_model_path = "invoice_model.onnx"
onnx_session = ort.InferenceSession(onnx_model_path)


In [17]:
def preprocess_text(text, tokenizer, max_length=512):
    encoded_input = tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return {
        'input_ids': encoded_input['input_ids'].numpy(),
        'attention_mask': encoded_input['attention_mask'].numpy(),
        'token_type_ids': encoded_input['token_type_ids'].numpy()
    }


In [21]:
def run_model(text, tokenizer, onnx_session):
    inputs = preprocess_text(text, tokenizer)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']

    # Prepare the input dictionary for ONNX
    input_feed = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'token_type_ids': token_type_ids
    }

    # Run the ONNX model
    outputs = onnx_session.run(None, input_feed)
    return outputs
